# 추천

### 최근접 이웃 협업 필터링
- COLLABORATIVE FILTERING:  잠재요인 협업 필터링 , 협업기반의 필터링
- CONTENT-BASED FILTERING: 컨텐츠(아이템) 기반의 추천,


- 최근접 이웃 협업 필터링
  - 아이템기반
  - 유저기반

- 최근접 이웃 협업 필터링
  - 사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식을 기반으로 추천
  - User-Item Matrix Concept
  - User사용자가 평가한 다른 아이템을 기반으로 사용자가 평가하지 않은 아이템의 예측 평가를 도출하는 방식
### 사용자 기반(User-User) 협업 필터링
- 나와 다른 사용자 간의 유사도를 측정하여 유사도가 높은 다른 사용자 N명을 추출하여 그들이 선호하는 아이템을 추천하는 방식

### 아이템 기반(item-item) 협업 필터링
- 나와 동일한 아이템을 사용해본 다른 사용자들과 평점 분포가 비슷한 다른 아이템을 추천하는 방식
- 일반적으로 사용자 기반보다는 아이템 가반 협업 필터링이 정확도가 더 높다.

- 잠재 요인 협업 필터링
  - 사용자_아이템 평저 메트릭스 속에 숨어 있는 잠재 요인을 추출해서 추천 예측을 하는 기법
  - 대규모 다차원 행렬인 사용자_아이템 행렬 데이터를 고윳값 분해와 같은 차원 감소 기법으로 분해하는 과정에서 잠재 요인을 추출한다.
  - 사용자 X 아이템 행렬 = (사용자 X 잠재요인 행렬) . (아이템 X 잠재요인 행렬).T 로 분해 후 두행렬의 내적을 통해 새로운 예측 사용자-아이템 평점 행렬 데이터를 생성하여 사용자가 아직 평점을 부여하지 않은 아이ㅔㅁ에 대한 예측 평점을 생성하는 것
  - `잠재요인`인 어떤것인지는 명확히 정의할 수 없다.
  - 만약, 영화 평점 기반의 사용자0아이템 행렬 데이터라면 영화가 가지는 장르별 특성 선호도로 가정할 수 있다.
  

### 특이값 분해(SVD) 싱귤러 벡터 디포지션
- SVD란 m x n 행렬일 떄 , 다음과 같이 3개의 행렬의 곱으로 분해하는 행렬 분해 방법 중 하나이다.
- SVD는 NaN값이 없는 행렬에만 적용할 수 있다. 하지만 R행렬에(시용자-영화 평점 행렬)는 아직 평점되지 않은 많은 Null값이 있기 떄문에 P와 Q행렬을 일반적인 SVD 방식으로 분해할 수 없다.
- 이러한 경우에는 확률적 경사 하강법 방식을 이용해 SVD를 수행한다.

kaggle
https://www.imdb.com/

# 장르 속성을 이용한 영화 콘텐츠 기반 필터링
- TMDB 5000 데이터 셋: 영화 데이터 정보 사이트인 IMDB.com의 영화 중 주요 영화 5000개에 대한 메타 정보를 가공해서 kaggle에서 제공하는 데이터 셋 
- https://www.kaggle.com/code

### 1.1 데이터 로딩 및 가공

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

movies = pd.read_csv('./tmdb_5000_movies.csv')
print(movies.shape)
movies.head(2)


(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


- 분석에 사용할 주요 컬럼 추출
  - id, title, genres, vote_average(평균 평점), vote_count(평점 투표수), popularity(영화 인기도), keyword, overview(영화 개요)

In [2]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [3]:
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."


In [4]:
movies_df['genres'].values[0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

### eval() 와 literal_eval() 의 차이
- eval        : 문자 형태로 되어있는 표현식을 실행하는 함수로, 함수나 객체도 실행가능
- literal_eval: 파이선에서 제공하는 기본 데이터 타입 정도만 변환해주는 용도로 사용 가능

- literal_eval() 함수를 통해 genres 와 keywords 컬럼의 값을 List객체로 변환

In [5]:
s = '(1+2) * 3'
eval(s)

9

In [6]:
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

- 장르와 키워드 컬럼의 값 중 name 키 값만 원소로 추출하여 리스트로 생성

In [7]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])

### `장르 컨텐츠 유사도 측정`
- 문자열로 변환된 'genres' 컬럼은 카운터 기반으로 피처 벡터화 변환 (사이킷 런의 CountVertorize 클래스 이용)
- 'genres' 문자열을 피처 벡터화 행렬로 변환한 데이터 셋을 코사인 유사도를 통해 비교
- 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천한다.

### `CountVertorize`
  - 텍스트에서 단위별 출현 횟수를 카운팅하여 수치 벡터화 한다.


In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1,1))
vectorizer.fit(['첫번째 문서 테스트','두번째 문서 테스트'])
# 학습한 문서의 고유한 어휘가 딕셔너리처럼 각각의 인덱스를 가지고 있다.
print(sorted(vectorizer.vocabulary_.items(),key=lambda x : x[1]))

# transform(): 문자열 목록을 전달하여 미리 학습 해놓은 사전을 기반으로 어휘의 빈도수를 세어준다.
counts = vectorizer.transform(['직접 첫번째 테스트 두번째 테스트'])
print(counts.toarray())

[('두번째', 0), ('문서', 1), ('첫번째', 2), ('테스트', 3)]
[[1 0 1 2]]


In [19]:
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x: ' '.join(x))
movies_df['genres_literal']

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genres_literal, Length: 4803, dtype: object

In [20]:
movies_df['genres_literal'][0]

'Action Adventure Fantasy Science Fiction'

In [28]:
#min df: 전체 문서에서 낮은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터
# 만약, min_df 2일 경우 전체 문서에 걸쳐 2개 이하로 나타나는 단어는 추출에서 제외한다.
# ngram_range: 모델의 단어 순서를 보강하기 위한 범위(범위 최소값, 범위 최대값)
count_vect = CountVectorizer(min_df=0,ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape) 

# 4803 -> 영화의개수
# 276  -> Countvectorizer에 의해 생성된 장르 어휘 갯수
print(genre_mat.toarray()[0])

(4803, 276)
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### 코사인 유사도(Cosine Similarity)
- 문서와 문서간의 유사도는 일반적으로 코사인 유사도를 사용한다
- 코사인 유사도는 벡터 간의 유사도를 비교할 때 벡터형 

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat,genre_mat)
print(genre_sim.shape)
print(genre_sim[:3])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]]


In [35]:
genre_sim_sorted_idx = genre_sim.argsort()[:,::-1]

# 첫 번째 영화와 장르 유사도가 높은 top10 영화의 movie_df 상에서 index값
# 앞에 0 인덱스는 자기 자신의 인덱스를 가리킨다(제외 필요)
print(genre_sim_sorted_idx[0][:10])

[   0 3494  813  870   46   14 1296 1652  419  420]


In [36]:
movies_df.columns

Index(['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity',
       'keywords', 'overview', 'genres_literal'],
      dtype='object')

### 1.3 장르 콘텐츠 필터링을 통한 영화 추천

In [46]:
# # title_name: 추천 기준이 되는 영화 제목(해당 영화와 장르 유사도가 높은 영화를 추천)
def find_sim_movie(df, sorted_idx,title_name, top_n=10):
  target_movie = df[df['title'] == title_name]
  title_index = target_movie.index.values
  
  similar_indexes = sorted_idx[title_index, :top_n]
  similar_indexes = similar_indexes.reshape(-1)

  return df.iloc[similar_indexes]


In [51]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_idx, 'The Godfather', 10)
similar_movies[['title','vote_average']]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [52]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average',ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- 외곡된 평점 데이터를 피하기 위해 평점에 평가 횟수를 반영한 가중 평점 방식 적용
- 가중 평점(Weighted Rating) = `((v/(v+m)) * R) + ((m/(v+m)) * C)`
  >- v: 개별 영화에 평점을 투표한 횟수
  >- m: 평점을 부여하기 위한 최소 투표 횟수 (전체 투표횟수에서 상위 60%에 해당하는 횟수를 기준으로 정함<임의 기준>)
  >- R: 개별 영화에 대한 평균평점
  >- C: 전체 영화에 대한 평균평점


In [53]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print(f'C:{C:.3f},m:{m:.3f},')

C:6.092,m:370.200,


In [55]:
def weighted_vote_average(record):
  v = record['vote_count']
  R = record['vote_count']
  
  return ((v/(v+m)) * R) + ((m/(v+m)) * C)

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [56]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
96,Inception,8.1,13391.664140,13752
65,The Dark Knight,8.2,11643.059385,12002
0,Avatar,7.2,11441.246267,11800
16,The Avengers,7.4,11417.268884,11776
788,Deadpool,7.4,10637.057009,10995
95,Interstellar,8.1,10509.196626,10867
287,Django Unchained,7.8,9742.106018,10099
94,Guardians of the Galaxy,7.9,9385.575772,9742
426,The Hunger Games,6.9,9098.978171,9455
127,Mad Max: Fury Road,7.2,9071.018691,9427


In [57]:
def find_sim_movie(df, sorted_idx, title_name, top_n = 10 ):
  title_movie = df[df['title'] == title_name]
  title_index = title_movie.index.values
  
  similar_indexs = sorted_idx[title_index,:(top_n*2)]
  similar_indexs = similar_indexs.reshape(-1)
  # 기준이 되는 영화 제외
  similar_indexs = similar_indexs[similar_indexs != title_index]
  
  return df.iloc[similar_indexs].sort_values('weighted_vote',ascending=False)[:top_n]

In [58]:
similar_movies = find_sim_movie(movies_df,genre_sim_sorted_idx,'The Godfather',10)
similar_movies[['title','vote_average','weighted_vote']]

,title,vote_average,weighted_vote
883,Catch Me If You Can,7.7,3458.244579
2731,The Godfather: Part II,8.3,3005.366302
1847,GoodFellas,8.2,2797.621440
1149,American Hustle,6.8,2480.644694
3866,City of God,8.1,1507.577750
281,American Gangster,7.4,1206.206240
1663,Once Upon a Time in America,8.2,795.592219
588,Wall Street: Money Never Sleeps,5.8,284.180169
2839,Rounders,6.9,240.949483
4041,This Is England,7.4,182.793674
